In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df= pd.read_csv('Churn_Modelling.csv')
X= df.iloc[:, 3:13]
Y= df.iloc[:, 13]



In [3]:
geography= pd.get_dummies(df['Geography'],drop_first=True)
gender= pd.get_dummies(df['Gender'],drop_first=True)



In [4]:
X= pd.concat([X,geography,gender],axis=1)
X= X.drop(['Geography','Gender'],axis=1)



In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(X,Y, test_size= 0.2, random_state= 0)

from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [8]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV


Using TensorFlow backend.


In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid


In [10]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [11]:
model = KerasClassifier(build_fn=create_model, verbose=0)


In [12]:
layers= [[20],[40,20],[50,20,15]]
activations= ['sigmoid','relu']
param_grid= dict(layers=layers, activation=activations, batch_size= [128,256], epochs=[30])
grid= GridSearchCV(estimator= model, param_grid= param_grid,cv=5)

In [13]:
grid_result= grid.fit(X_train,Y_train)


In [21]:
[grid_result.best_score_,grid_result.best_params_]


[0.8555000066757202,
 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [40, 20]}]

In [22]:
y_pred= grid.predict(X_test)
y_pred= (y_pred >0.5)

In [23]:
from sklearn.metrics import confusion_matrix,accuracy_score

cm= confusion_matrix(y_pred,Y_test)
cm

array([[1557,  227],
       [  38,  178]])

In [25]:
accuracy= accuracy_score(y_pred,Y_test)
accuracy

0.8675

In [26]:
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


NameError: name 'model_history' is not defined